In [1]:
import findspark
findspark.init()

In [153]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import isnan, when, count, col, length, size

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Page View") \
    .getOrCreate()

In [13]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [29]:
file = './Data/Mid_Data/online_pageviews.json'

In [5]:
## Cria Spark DF
df = spark.read.json(file)

# 1.0 - Análise Exploratória de Dados

In [7]:
df.show(10)

+-----------+-----------+----------+----------+---------------+------------+--------------------+
|category_id|customer_id|      date|deviceType|  on_product_id|    pageType|          visitor_id|
+-----------+-----------+----------+----------+---------------+------------+--------------------+
|       null|       null|2018-08-01|   desktop|           null|        home|61623262303037613...|
|       null|       null|2018-08-01|   desktop|           null|landing_page|35383630323633313...|
| 6161393336|       null|2018-08-01|    mobile|           null| subcategory|31303766393633366...|
| 6664353638|       null|2018-08-01|    mobile|           null| subcategory|31303766393633366...|
|       null|       null|2018-08-01|   desktop|           null|landing_page|32353739633733323...|
| 6465643262|       null|2018-08-01|   desktop|373662303135636|     product|39356362316264336...|
| 3832383865|       null|2018-08-01|    mobile|           null| subcategory|63383335343461373...|
|       null|       

In [26]:
## Verificar Colunas
df.printSchema()

root
 |-- category_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- deviceType: string (nullable = true)
 |-- on_product_id: string (nullable = true)
 |-- pageType: string (nullable = true)
 |-- visitor_id: string (nullable = true)



In [8]:
## Verificar Qual dispositivo é mais utilizado para acessos
df.groupby('deviceType').count().show()

+----------+-------+
|deviceType|  count|
+----------+-------+
|   desktop|1353749|
|    mobile|2098791|
+----------+-------+



**Condiz com o visto no DataSet de compras**

In [19]:
## Verificar Valores Nulos nas Colunas
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+-----------+----+----------+-------------+--------+----------+
|category_id|customer_id|date|deviceType|on_product_id|pageType|visitor_id|
+-----------+-----------+----+----------+-------------+--------+----------+
|    1014834|    3371775|   0|         0|      2047370|       0|         0|
+-----------+-----------+----+----------+-------------+--------+----------+



**Coluna com mais valores nulos = customer_id**  
**on_product_id provavelmente só seja diferente de nulo quando pageType = product**  
**Todo visitante tem uma identificação** 

In [156]:
## Verificar Tipos de Páginas e Quais são mais acessadas
devices = df.groupby('pageType').count()
devices.show()

+------------+-------+
|    pageType|  count|
+------------+-------+
|landing_page| 275505|
| userprofile|   2058|
|  topqueries|    117|
| emptysearch|  29618|
|confirmation|   8351|
|       other|  75302|
|        cart|  30203|
|   not_found|  87938|
| subcategory| 689586|
|      search|  94510|
|    category| 344513|
|        home| 381654|
|    checkout|  17332|
|     product|1415853|
+------------+-------+



# 2.0 - Criar DataFrames Auxiliares no Pandas

## 2.1 - Páginas

In [157]:
## Cria Pandas DF com as páginas
df_dev = devices.select("*").toPandas()

In [159]:
df_dev.head(20)

,pageType,count
0,landing_page,275505
1,userprofile,2058
2,topqueries,117
3,emptysearch,29618
4,confirmation,8351
5,other,75302
6,cart,30203
7,not_found,87938
8,subcategory,689586
9,search,94510


## 2.2 - Lista de Clientes Identificados que vizualizaram alguma página

In [151]:
## Lista de Clientes Identificados Spark
clients_sp = df.groupby(['customer_id']).count()
clients_sp.show(5)

+--------------------+-----+
|         customer_id|count|
+--------------------+-----+
|35613338323466366463|   63|
|62643039356362363064|  475|
|35363961316461323864|   38|
|63303038613765313230|   18|
|31663662386165343935|    7|
+--------------------+-----+
only showing top 5 rows



In [38]:
## Data Frame com clientes identificados e número de vezes que acessaram alguma pagina
df_clients = clients_sp.select("*").toPandas()

In [44]:
df_clients.head()

,customer_id,count
0,35613338323466366463,63
1,62643039356362363064,475
2,35363961316461323864,38
3,63303038613765313230,18
4,31663662386165343935,7


In [75]:
## Pega indice para dropar linha com valores None pois esse agrupamento nao diz nada pela falta de ID
index_None = list(df_clients[df_clients['count']==df_clients['count'].max()].index)[0]

In [78]:
## Dropa o valor None da lista, usar essa lista de clientes para verificar nas compras offline
df_clients.drop([index_None],axis=0, inplace=True)

In [82]:
## Cliente com Maior frequencia de acessos
ma_client = df_clients[df_clients['count']==df_clients['count'].max()]
ma_client

,customer_id,count
4627,63623261623161633734,1122


In [86]:
len(df_clients.customer_id)

5913

## 2.3 - Lista de Produtos Acessados

In [88]:
products_sp = df.groupby(['on_product_id']).count()

In [109]:
## Data Frame de Produtos e frequencia de visualização
products = products_sp.select("*").toPandas()

In [133]:
products.head()

,on_product_id,count
0,393038626266336,237
1,623332626333363,1059
2,633937373266613,1676
3,386463646465306,386
4,363166646432643,786


In [110]:
len(products)

3189

In [115]:
## Pega indice para dropar linha com valores None pois esse agrupamento nao diz nada pela falta de ID
index_none = list(products[products['count']==products['count'].max()].index)[0]

In [116]:
## Dropa Linha
products.drop([index_none],axis=0, inplace=True)

In [118]:
## Produto Mais Visualizado
ma_product = products[products['count']==products['count'].max()]

## 2.4 - Lista com IDs de visitantes e produto visualizado

In [166]:
## Filtra DataFrame para mostrar apenas acesso à página do tipo PRODUTO
df_filtered = df.filter(df['pageType']=='product')
df_filtered.show(10)

+-----------+-----------+----------+----------+---------------+--------+--------------------+
|category_id|customer_id|      date|deviceType|  on_product_id|pageType|          visitor_id|
+-----------+-----------+----------+----------+---------------+--------+--------------------+
| 6465643262|       null|2018-08-01|   desktop|373662303135636| product|39356362316264336...|
| 3834323666|       null|2018-08-01|    mobile|616264656437353| product|31376330316664366...|
| 3363396661|       null|2018-08-01|   desktop|373830663266653| product|64353563636165383...|
| 3166373239|       null|2018-08-01|   desktop|356566356434626| product|61636236386561663...|
| 3436343737|       null|2018-08-01|    mobile|626334343231306| product|32396331656334646...|
| 3232653962|       null|2018-08-01|    mobile|376235663566663| product|32323133633730333...|
| 3762363534|       null|2018-08-01|   desktop|373330303834353| product|30636434643732396...|
| 3331663330|       null|2018-08-01|   desktop|3464643633393

In [173]:
## Agrupa por id e produto 
visitors_sp = df_filtered.groupby('visitor_id','on_product_id').count()
visitors_sp.show(10)

+--------------------+---------------+-----+
|          visitor_id|  on_product_id|count|
+--------------------+---------------+-----+
|62653931313965336...|643564616339663|    2|
|33386664653035636...|653332626234666|    1|
|62326233343963656...|623361636538373|    1|
|32653965323334653...|306530623037386|    1|
|63356437343837613...|663162653735336|    1|
|31306636396331386...|613638653161613|    1|
|39636533396131316...|626334343231306|    3|
|38643735366261326...|613266396338323|    1|
|36346630356532643...|666534626430323|    1|
|31616562633162353...|306237393062326|    1|
+--------------------+---------------+-----+
only showing top 10 rows



In [174]:
## Cria DF no Pandas para melhor manipuação
visitors = visitors_sp.select("*").toPandas()

In [181]:
## Dropa Linhas com valores nulos de on_product_id (sem filtro pois visitor_id nao tem valor nulo)
visitors.dropna(inplace=True)

In [197]:
## Pandas DataFrame contendo ID visitante e produto acessado
visitors_grouped = visitors.groupby(['visitor_id','on_product_id']).sum().reset_index()
visitors_grouped.head()

,visitor_id,on_product_id,count
0,3030303031303364643064326,363966653132636,1
1,3030303031303364643064326,643733373961386,1
2,3030303031303364643064326,653466303735636,1
3,3030303032333232333334633,626538323635626,1
4,3030303063396466356636393,623166353632633,1


### Utilizar esse dataframe para comparar com o dataframe de compras para criar uma lista de quem não comprou

In [198]:
## Exporta DataFrame de Saída em Colunas
visitors_grouped.to_csv(r'./Outputs/Utils/visitors_grouped.csv')